In [1]:
import osmnx as ox, networkx as nx, matplotlib.cm as cm, pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import geopandas as gpd
import functools
import community

from scipy import sparse
from scipy.sparse import linalg
import time
from shapely.geometry import Point, LineString, Polygon, MultiPolygon, mapping
from math import sqrt
import pandas as pd
from shapely.ops import cascaded_union
pd.set_option('precision', 5)
pd.options.display.float_format = '{:20.2f}'.format

In [5]:
buildings.to_file('Outputs/bh_p.shp', driver='ESRI Shapefile')

In [2]:
buildings = gpd.read_file('C:/Users/gabri/Google Drive Academic/GIS/Building_Arc.shp')

In [3]:
buildings.head()

,BUILDGTHEM,CLASSIFICA,CLASSIFI_1,DISTNAME,DISTNAME_1,ID,Important_,NEAR_DIST,Neighbours,OBJECTID,geometry,max,type
0,,,,,,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,0,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,residential
1,,,,,,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,0,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,residential
2,,,,,,33087D9B-5E09-489C-85AD-1A0A74B553CC,0,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,
3,,,,,,E3A12B52-5690-48B6-9ED7-8124856C68C4,0,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,
4,,,,,,60941772-56BC-43A7-9C92-45523E6A137B,0,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,


In [4]:
buildings = buildings.drop(['DISTNAME','DISTNAME_1'], axis=1)

In [5]:
buildings.columns = ['Main_func', 'Second_func', 'Station_Type', 'OS_id', 'Important', 'Closest_Road_Distance', 
                     'Neighbours', 'Building_id', 'geometry', 'height_max','OSM_use']

In [6]:
#computing Area
buildings["area"] = buildings['geometry'].area

In [7]:
#land use

POI = gpd.read_file('Outputs/POI.shp')
POI['nr']=1

In [8]:

for row in buildings.itertuples():
    g = row[9] #geometry
    
    sindex = POI.sindex
    possible_matches_index = list(sindex.intersection(g.bounds))
    possible_matches = POI.iloc[possible_matches_index]
    
    if (len(possible_matches)==0): continue
    else:
        use = possible_matches.groupby(['main'],as_index=False)['nr'].sum().sort_values(by='nr',
                                    ascending=False).reset_index()
        main_use = use['main'].loc[0]
        buildings.set_value(row[0], 'POI_main', main_use)

In [9]:
#filling empty spaces with NaN
buildings = buildings.apply(lambda x: x.str.strip() if isinstance(x, str) else x).replace('', np.nan)
buildings.head()

,Main_func,Second_func,Station_Type,OS_id,Important,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,OSM_use,area,POI_main
0,NaN,NaN,NaN,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,0,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,residential,386.30,NaN
1,NaN,NaN,NaN,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,0,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,residential,2445.51,Commercial services
2,NaN,NaN,NaN,33087D9B-5E09-489C-85AD-1A0A74B553CC,0,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,NaN,3602.66,Commercial services
3,NaN,NaN,NaN,E3A12B52-5690-48B6-9ED7-8124856C68C4,0,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,NaN,1849.65,Attractions
4,NaN,NaN,NaN,60941772-56BC-43A7-9C92-45523E6A137B,0,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,NaN,1454.43,NaN


In [10]:
buildings.POI_main.fillna(buildings.OSM_use, inplace=True)

In [11]:
buildings.POI_main.fillna('residential', inplace=True)

In [12]:
buildings.Main_func.fillna(buildings.POI_main, inplace=True)

In [13]:
buildings.head()

,Main_func,Second_func,Station_Type,OS_id,Important,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,OSM_use,area,POI_main
0,residential,NaN,NaN,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,0,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,residential,386.30,residential
1,Commercial services,NaN,NaN,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,0,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,residential,2445.51,Commercial services
2,Commercial services,NaN,NaN,33087D9B-5E09-489C-85AD-1A0A74B553CC,0,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,NaN,3602.66,Commercial services
3,Attractions,NaN,NaN,E3A12B52-5690-48B6-9ED7-8124856C68C4,0,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,NaN,1849.65,Attractions
4,residential,NaN,NaN,60941772-56BC-43A7-9C92-45523E6A137B,0,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,NaN,1454.43,residential


In [14]:
buildings_tmp = buildings[['Main_func', 'OS_id', 'Closest_Road_Distance',
                          'Neighbours','Building_id','geometry','height_max','area']].copy()

In [15]:
buildings_tmp.rename(columns={'Main_func': 'Land_Use'}, inplace=True)

In [85]:
list(buildings_tmp['Land_Use'].unique())

['Residential',
 'Commercial services',
 'Attraction',
 'Manufacturing and production',
 'Accommodation, eating and drinking',
 'Public infrastructure',
 'Sport and entertainment',
 'Education',
 'Religious Buildings',
 'Emergency Service',
 'Transport',
 'Medical Care',
 'construction',
 'industrial',
 'Cultural Facility',
 'military']

In [17]:
buildings_tmp['Land_Use'] = buildings_tmp['Land_Use'].map( lambda x: 'Residential' if x == 'residential'
                                                          else 'Residential' if x == 'grass'
                                                          else 'Commercial services' if x == 'retail'
                                                          else 'Commercial services' if x == 'Retail' 
                                                          else 'Commercial services' if x == 'commercial' 
                                                          else 'Commercial services' if x == 'market'
                                                          else 'Education' if x == 'Education and health'
                                                          else 'Transport' if x == 'Road Transport'
                                                          else 'Transport' if x =='railway'
                                                          else 'Attraction' if x =='Attraction And Leisure'
                                                          else 'Attraction' if x =='Attractions'
                                                          else 'Sport and entertainment' if x == 'Sports Or Exercise Facility'
                                                          else x)

In [18]:
#computing Envelope polygon

for index, row in buildings_tmp.iterrows():
    t = row['geometry'].envelope
    coords = mapping(t)['coordinates'][0]
    d = [(Point(coords[0])).distance(Point(coords[1])), (Point(coords[1])).distance(Point(coords[2]))]

    width = min(d)
    length = max(d)
    
    buildings_tmp.set_value(index, 'width', width)
    buildings_tmp.set_value(index, 'length', length)

In [19]:
buildings_tmp['extension']=buildings_tmp.area*(buildings_tmp.length/buildings_tmp.width)

In [20]:
buildings_tmp['facade_area']=buildings_tmp['height_max']*(buildings_tmp.width)

In [21]:
buildings_tmp.head()

,Land_Use,OS_id,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,area,width,length,extension,facade_area
0,Residential,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,386.30,18.79,47.74,2274.28,337.28
1,Commercial services,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,2445.51,73.36,109.86,13442.55,2062.46
2,Commercial services,33087D9B-5E09-489C-85AD-1A0A74B553CC,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,3602.66,110.64,147.26,19302.04,2081.83
3,Attraction,E3A12B52-5690-48B6-9ED7-8124856C68C4,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,1849.65,51.09,75.41,9590.18,1408.04
4,Residential,60941772-56BC-43A7-9C92-45523E6A137B,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,1454.43,46.84,48.90,7160.71,256.68


In [22]:
buildings_tmp = buildings_tmp[buildings_tmp['area']>199]

In [23]:
len(buildings_tmp)

4219

In [24]:
all_build = gpd.read_file("C:/Users/gabri/Google Drive Academic/GIS/TQ_small.shp")

In [25]:
copy = all_build.copy()

In [26]:
buildings_tmp['prominence']='NaN'

In [27]:
for row in buildings_tmp.itertuples():
    
    g = row[6] #geometry
    fil = g.buffer(120)
    OS_index = row[2]
    copy_tmp = copy.loc[copy.ID != OS_index]
    
    sindex = copy_tmp.sindex
    possible_matches_index = list(sindex.intersection(fil.bounds))
    possible_matches = copy_tmp.iloc[possible_matches_index]
    
    polygon = MultiPolygon([pol for pol in possible_matches['geometry']])
    
    i=2
    try:
        while(g.buffer(i).intersects(polygon)==False): 
            i=i+2
    except:
        while(g.buffer(i).distance(polygon)>0):
            i=i+2
    
    area_max = (g.buffer(i-2)).area-(g.area)
    if (area_max < 0):
        area_max = 0

    buildings_tmp.set_value(row[0], 'prominence', area_max)

TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 182511.45662080735
Self-intersection at or near point 529260.30993121245 182511.45662080735
TopologyException: side location conflict at 529260.30993121245 

In [28]:
buildings_tmp.head()

,Land_Use,OS_id,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,area,width,length,extension,facade_area,prominence
0,Residential,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,386.30,18.79,47.74,2274.28,337.28,2572.93
1,Commercial services,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,2445.51,73.36,109.86,13442.55,2062.46,2482.91
2,Commercial services,33087D9B-5E09-489C-85AD-1A0A74B553CC,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,3602.66,110.64,147.26,19302.04,2081.83,3644.26
3,Attraction,E3A12B52-5690-48B6-9ED7-8124856C68C4,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,1849.65,51.09,75.41,9590.18,1408.04,4673.65
4,Residential,60941772-56BC-43A7-9C92-45523E6A137B,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,1454.43,46.84,48.90,7160.71,256.68,3974.24


In [29]:
#load historical points

historic = gpd.read_file("C:/Users/gabri/Google Drive Academic/GIS/Historic England/Listed_building_selection.shp")

In [30]:
historic.head()

,AmendDate,CaptureSca,Easting,Grade,LegacyUID,ListDate,ListEntry,Location,NGR,Name,Northing,geometry
0,None,1:2500,532101.61,II*,466565,1997-12-04,1021941.00,"City of London, London, EC1M",TQ 32102 82099,CRESCENT HOUSE INCLUDING GROUND FLOOR SHOPS AN...,182099.36,POINT (532101.6059999997 182099.3577999994)
1,None,1:2500,532118.00,II,466566,1997-12-04,1021942.00,"City of London, London, EC1M",TQ 32118 82169,HATFIELD HOUSE INCLUDING GARDEN WALLS TO REAR,182169.36,POINT (532118 182169.3607999999)
2,None,1:2500,532203.00,II,466567,1997-12-04,1021943.00,"City of London, London, EC1Y",TQ 32203 82152,BASTERFIELD HOUSE INCLUDING STEPS TO GARDEN,182152.36,POINT (532203 182152.3607999999)
3,None,1:2500,532231.00,II,466568,1997-12-04,1021944.00,"City of London, London, EC1Y",TQ 32231 82115,BAYER HOUSE AND RAISED PAVEMENTS TO NORTH AND ...,182115.36,POINT (532231 182115.3607999999)
4,None,1:2500,532194.00,II,466569,1997-12-04,1021945.00,"City of London, London, EC1M",TQ 32194 82073,GREAT ARTHUR HOUSE INCLUDING BOILER HOUSE,182073.36,POINT (532194 182073.3607999999)


In [31]:
historic=historic[['Grade','geometry']]

In [32]:
historic['Grade'] = historic['Grade'].map( lambda x: 3 if x == 'I' else 2 if x == 'II*' else 1 
                                   if x == 'II' else 0)

In [51]:
for row in buildings_tmp.itertuples():
    g = row[6] #geometry
    
    sindex = historic.sindex
    possible_matches_index = list(sindex.intersection(g.bounds))
    possible_matches = historic.iloc[possible_matches_index]
    if (len(possible_matches)==0): buildings_tmp.set_value(row[0], 'historic_meaning', 0)
    else: buildings_tmp.set_value(row[0], 'historic_meaning', possible_matches['Grade'].sum())

In [52]:
buildings_tmp.head()

,Land_Use,OS_id,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,area,width,length,extension,facade_area,prominence,pragmatic_meaning,historic_meaning
0,Residential,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,386.30,18.79,47.74,2274.28,337.28,2572.93,0.74,0.00
1,Commercial services,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,2445.51,73.36,109.86,13442.55,2062.46,2482.91,0.63,1.00
2,Commercial services,33087D9B-5E09-489C-85AD-1A0A74B553CC,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,3602.66,110.64,147.26,19302.04,2081.83,3644.26,0.56,0.00
3,Attraction,E3A12B52-5690-48B6-9ED7-8124856C68C4,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,1849.65,51.09,75.41,9590.18,1408.04,4673.65,0.97,0.00
4,Residential,60941772-56BC-43A7-9C92-45523E6A137B,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,1454.43,46.84,48.90,7160.71,256.68,3974.24,0.70,0.00


In [53]:
copy = buildings_tmp.copy()
copy['nr']=1

In [36]:
# neigh = possible_matches.groupby(['Land_Use'],as_index=False)['nr'].sum()
# neigh['Pj']=neigh['nr']/neigh['nr'].sum()
# neigh['i']=neigh['Pj']*np.log2(neigh['Pj'])
# neigh.set_index('Land_Use', inplace=True)
# Entropy =-neigh.i.sum()

In [54]:
for row in buildings_tmp.itertuples():
    g = row[6] #geometry
    b = g.buffer(200)
    use = row[1]
    
    sindex = copy.sindex
    possible_matches_index = list(sindex.intersection(b.bounds))
    possible_matches = copy.iloc[possible_matches_index]
        
    neigh = possible_matches.groupby(['Land_Use'],as_index=True)['nr'].sum()
    Nj = neigh.loc[use]
    #Pj = Nj/N
    Pj = 1-(Nj/possible_matches['nr'].sum())
    buildings_tmp.set_value(row[0], 'pragmatic_meaning', Pj)

In [55]:
buildings_tmp.sort_values(by='pragmatic_meaning', ascending=False).head()

,Land_Use,OS_id,Closest_Road_Distance,Neighbours,Building_id,geometry,height_max,area,width,length,extension,facade_area,prominence,pragmatic_meaning,historic_meaning
2901,Cultural Facility,227BD424-10E3-491B-98B6-7EE06773ED5E,0.00,19,3514,"POLYGON ((533243.2400000002 179571.25, 533271....",3.04,19899.65,197.36,263.83,235465.88,600.96,8448.32,0.99,3.00
3236,Cultural Facility,2D5A9164-BA18-42F3-A648-50EFF8D4FA7F,0.00,21,3939,"POLYGON ((529640.4299999997 178449.0299999993,...",24.31,11157.85,259.12,311.73,68551.02,6298.69,9207.88,0.99,33.00
627,Sport and entertainment,8498A3C6-21EF-4F3D-B2E7-844CFFC94C50,0.00,22,800,"POLYGON ((532156.9100000001 182139.9700000007,...",25.17,7620.13,166.88,217.25,61958.18,4200.37,5328.98,0.99,12.00
346,Transport,B8B49B4B-66F2-42CA-A034-4DAD63A0BFB4,0.00,16,461,"POLYGON ((530976.3600000003 182539.7200000007,...",15.98,2647.03,127.49,149.56,9713.31,2037.29,945.13,0.99,3.00
3566,Emergency Service,5FA2DAEC-45DA-418B-9B84-D9665D770470,1.23,8,4363,"POLYGON ((532330.7800000003 179557.6500000004,...",19.73,4152.22,87.63,131.96,24269.91,1728.68,1026.12,0.99,1.00


In [56]:
visibility = pd.read_csv('visibility.csv')

In [57]:
visibility.head()

,Unnamed: 0,Building_id,distance,mean_distance,n_lines
0,0,2,356.40,257.30,23
1,1,3,564.76,203.91,6
2,2,5,151.18,131.50,3
3,3,9,186.19,68.25,8
4,4,11,259.08,246.18,5


In [58]:
tmp = pd.merge(buildings_tmp,visibility[['Building_id','distance', 'n_lines','mean_distance']], on='Building_id', how='left')

In [59]:
tmp['prominence']=tmp['prominence'].astype(float)

In [60]:
tmp['distance'].fillna((tmp['distance'].min()), inplace=True)
tmp['mean_distance'].fillna((tmp['mean_distance'].min()), inplace=True)
tmp['n_lines'].fillna((tmp['n_lines'].min()), inplace=True)

In [44]:
tmp.dtypes

Land_Use                  object
OS_id                     object
Closest_Road_Distance    float64
Neighbours                 int64
Building_id                int64
geometry                  object
height_max               float64
area                     float64
width                    float64
length                   float64
extension                float64
facade_area              float64
prominence               float64
historical_meaning       float64
historic_meaning         float64
pragmatic_meaning        float64
distance                 float64
n_lines                  float64
mean_distance            float64
dtype: object

In [61]:
tmp.rename(columns={'distance': 'sight_line', 'Closest_Road_Distance':'road_distance',
                   'Land_Use':'land_use','Building_id':'building_id', 'height_max':'height',
                   'Neighbours':'neighbours', 'historical_meaning':'historic_meaning','prominence':'openness'}, inplace=True)

In [47]:
tmp.head()

,land_use,OS_id,road_distance,neighbours,building_id,geometry,height,area,width,length,...,historic_meaning,pragmatic_meaning,sight_line,n_lines,mean_distance,area_scaled,extension_scaled,facade_area_scaled,height_scaled,openness_scaled
0,Residential,A87A27ED-5F80-4FDD-8194-31F6A5F1D84A,11.68,7,1,"POLYGON ((532680.6500000004 182920.5399999991,...",17.95,386.30,18.79,47.74,...,nan,0.74,18.57,1.00,13.79,0.00,0.00,0.01,0.06,0.07
1,Commercial services,FAE2A2F8-65DB-45B7-9F91-11019E87E52A,2.06,15,2,"POLYGON ((532361.5700000003 182971.6199999992,...",28.11,2445.51,73.36,109.86,...,1.00,0.63,356.40,23.00,257.30,0.03,0.01,0.07,0.10,0.06
2,Commercial services,33087D9B-5E09-489C-85AD-1A0A74B553CC,3.19,16,3,"POLYGON ((532438.1600000001 182978.3300000001,...",18.82,3602.66,110.64,147.26,...,nan,0.56,564.76,6.00,203.91,0.04,0.01,0.07,0.07,0.09
3,Attraction,E3A12B52-5690-48B6-9ED7-8124856C68C4,1.52,11,4,"POLYGON ((532619.1699999999 182949.6099999994,...",27.56,1849.65,51.09,75.41,...,nan,0.97,18.57,1.00,13.79,0.02,0.01,0.05,0.10,0.12
4,Residential,60941772-56BC-43A7-9C92-45523E6A137B,4.40,8,5,"POLYGON ((532551.9100000001 182930.5999999996,...",5.48,1454.43,46.84,48.90,...,nan,0.70,151.18,3.00,131.50,0.02,0.00,0.01,0.02,0.10


In [62]:
col = ['area', 'extension', 'facade_area', 'height', 'openness','historic_meaning', 'sight_line']

for i in col:
       tmp[i+"_scaled"] = (tmp[i]-tmp[i].min())/(tmp[i].max()-tmp[i].min())

In [63]:
col = ['neighbours', 'road_distance']

for i in col:
       tmp[i+"_scaled"] = 1-(tmp[i]-tmp[i].min())/(tmp[i].max()-tmp[i].min())

In [64]:
tmp_scaled = tmp[['building_id','geometry','area_scaled','extension_scaled','facade_area_scaled',
                 'height_scaled','openness_scaled','historic_meaning_scaled', 'sight_line_scaled',
                 'neighbours_scaled','road_distance_scaled','pragmatic_meaning']].copy()

In [65]:
tmp_scaled['visual']=(tmp_scaled['facade_area_scaled']*30+tmp_scaled['height_scaled']*20+
                      tmp_scaled['sight_line_scaled']*50)/100

In [66]:
tmp_scaled['structural']=(tmp_scaled['extension_scaled']*30+tmp_scaled['neighbours_scaled']*20+
                   tmp_scaled['openness_scaled']*30+tmp_scaled['road_distance_scaled']*20)/100

In [76]:
col = ['visual', 'structural', 'pragmatic_meaning']

for i in col:
       tmp_scaled[i+"_scaled"] = (tmp_scaled[i]-tmp_scaled[i].min())/(tmp_scaled[i].max()-tmp_scaled[i].min())

In [77]:
weights = tmp_scaled[['building_id','geometry','visual_scaled','structural_scaled',
                      'historic_meaning_scaled', 'pragmatic_meaning_scaled']].copy()

In [78]:
weights.head()

,building_id,geometry,visual_scaled,structural_scaled,historic_meaning_scaled,pragmatic_meaning_scaled
0,1,"POLYGON ((532680.6500000004 182920.5399999991,...",0.02,0.35,0.00,0.72
1,2,"POLYGON ((532361.5700000003 182971.6199999992,...",0.08,0.35,0.01,0.60
2,3,"POLYGON ((532438.1600000001 182978.3300000001,...",0.09,0.36,0.00,0.52
3,4,"POLYGON ((532619.1699999999 182949.6099999994,...",0.04,0.42,0.00,0.98
4,5,"POLYGON ((532551.9100000001 182930.5999999996,...",0.02,0.41,0.00,0.68


In [79]:
weights['score']=(weights['visual_scaled']*50+weights['structural_scaled']*30+weights['historic_meaning_scaled']*10+
                  weights['pragmatic_meaning_scaled']*10)/100

In [80]:
weights['score_scaled']=(weights['score']-weights['score'].min())/(weights['score'].max()-weights['score'].min())

In [81]:
weights.rename(columns={'visual_scaled': 'visual',
                        'structural_scaled':'structural',
                        'historic_meaning_scaled': 'historic',
                        'pragmatic_meaning_scaled':'pragmatic'}, inplace=True)

In [82]:
weights[['building_id','visual','structural','historic','pragmatic','score_scaled']].head(10)

,building_id,visual,structural,historic,pragmatic,score_scaled
0,1,0.02,0.35,0.00,0.72,0.20
1,2,0.08,0.35,0.01,0.60,0.23
2,3,0.09,0.36,0.00,0.52,0.23
3,4,0.04,0.42,0.00,0.98,0.29
4,5,0.02,0.41,0.00,0.68,0.23
5,6,0.01,0.33,0.00,0.61,0.17
6,9,0.05,0.40,0.00,0.52,0.21
7,10,0.01,0.33,0.00,0.98,0.23
8,11,0.04,0.33,0.00,0.61,0.20
9,12,0.02,0.40,0.00,0.66,0.22


In [83]:
weights.crs = {'init': 'epsg:27700'}

In [84]:
weights.to_file(driver='ESRI Shapefile',filename="Outputs/landmark_scores.shp")